#Analisis y Mineria de datos - Web Scrapping Box Office Mojo

Autor: Martin Nicolas Serafini


El scraping web es la técnica de extraer datos de sitios web de manera automatizada. En este cuaderno de Google Colab, que es un entorno de desarrollo de Python en la nube, podemos utilizar diferentes bibliotecas como **BeautifulSoup** y **Request**s para realizar esta tarea cuando la fuente es una pagina web.

* **request**s envía una solicitud a una URL y obtiene el código HTML de la página.
* **BeautifulSoup** toma ese código HTML, lo estructura en un formato navegable y permite buscar y extraer los datos específicos que nos interesan (en este caso, los datos de taquilla de la web).

Una vez extraídos los datos, se organizan y se almacenan en un formato estructurado, como un archivo CSV o un DataFrame de Pandas, para facilitar su análisis y visualización.

Tambien se puede utilizar la libreria **Pandas** para extraer tablas de HTML mediante **read_html(url)** que es un scraper automático de tablas HTML que transforma directamente las tablas de una página web en DataFrames de pandas, sin necesidad de usar BeautifulSoup ni expresiones regulares.

Finalmente se complementa con scrapping realizado sobre archivos .PDF mediante el uso de la libreria **PDFPlumber**.

In [ ]:
# Importa a instala las librrias necesarias
!pip install beautifulsoup4
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import requests
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
import re

In [ ]:
# Monto Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Web Scrapping - Ranking 200 peliculas mas vistas por año desde 1977 a la fecha

Fuente de los datos: https://www.boxofficemojo.com/

Tarea:
* Webscrapping del top 200 de peliculas desde el año 1977 a la actualidad.
* Transformacion y limpieza de datos
* Confeccion del dataset

In [ ]:
ranking_cine = []
año = datetime.datetime.now().year
# Scrapea los rankings desde 1977 hasta el año inmediato anterior a la consulta (2024 a fecha de subida)
for i in range (1977,año):
    url_año = f"https://www.boxofficemojo.com/year/world/{i}/"

    # Realizar la solicitud HTTP
    response_año = requests.get(url_año)

    # Analizar el contenido HTML
    soup_año = BeautifulSoup(response_año.text, 'html.parser')

    # Buscar la tabla de debilidades
    table_año = soup_año.find('table')

    # Extraer los datos de la tabla
    rows_año = table_año.find_all('tr')

    # Procesar los datos de la tabla

    for row in rows_año:
        columns = row.find_all('td')
        data = [col.text for col in columns]
        data.insert(0,i)
        ranking_cine.append(data)

# Genero los nombres de las columnas
ranking_total_cols = ['Year','Rank','Release Group','Worldwide','Domestic','Domestic %','Foreign','Foreign %']
# Genero el dataset con los datos scrapeados y los nombre de las columnas
dfranking_cine = pd.DataFrame(ranking_cine, columns = ranking_total_cols)

dfranking_cine

,Year,Rank,Release Group,Worldwide,Domestic,Domestic %,Foreign,Foreign %
0,1977,None,None,None,None,None,None,None
1,1977,1,Star Wars: Episode IV - A New Hope,"$307,263,857","$307,263,857",100%,-,-
2,1977,2,Smokey and the Bandit,"$126,748,082","$126,737,428",100%,$87,<0.1%
3,1977,3,Close Encounters of the Third Kind,"$116,396,241","$116,395,460",100%,-,-
4,1977,4,Saturday Night Fever,"$94,214,336","$94,213,184",100%,-,-
...,...,...,...,...,...,...,...,...
8892,2024,196,Io e te dobbiamo parlare,"$9,886,115",-,-,"$9,886,115",100%
8893,2024,197,Listy do M. Pozegnania i powroty,"$9,782,064",-,-,"$9,782,064",100%
8894,2024,198,Rafadan Tayfa: Kapadokya,"$9,514,264",-,-,"$9,514,264",100%
8895,2024,199,The Chosen: S4 Episodes 4-6,"$9,482,744","$9,482,744",100%,-,-


In [ ]:
# Verifico los tipos de las columnas, hay valores numericos como Object
dfranking_cine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8897 entries, 0 to 8896
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           8897 non-null   int64 
 1   Rank           8849 non-null   object
 2   Release Group  8849 non-null   object
 3   Worldwide      8849 non-null   object
 4   Domestic       8849 non-null   object
 5   Domestic %     8849 non-null   object
 6   Foreign        8849 non-null   object
 7   Foreign %      8849 non-null   object
dtypes: int64(1), object(7)
memory usage: 556.2+ KB


In [ ]:
# Depuro de valores NULL y genero un dataset depurado
# Verifica cuantos registro hay con valores null
null_recuento = dfranking_cine.isnull().sum()
print("Valores NULL por Columna:")
print(null_recuento)
# Muestra registros con datos null, el scrapping incluye una linea vacia por cada periodo scrapeado
registros_con_nulls = dfranking_cine[dfranking_cine.isnull().any(axis=1)]
print("\nRegistros con al menos 1 valor NULL:")
print(registros_con_nulls)
# Remueve columnas con datos null
dfranking_cine_depurado = dfranking_cine.dropna()
print("\nDataFrame depurado:")
dfranking_cine_depurado

Valores NULL por Columna:
Year              0
Rank             48
Release Group    48
Worldwide        48
Domestic         48
Domestic %       48
Foreign          48
Foreign %        48
dtype: int64

Registros con al menos 1 valor NULL:
      Year  Rank Release Group Worldwide Domestic Domestic % Foreign Foreign %
0     1977  None          None      None     None       None    None      None
40    1978  None          None      None     None       None    None      None
97    1979  None          None      None     None       None    None      None
182   1980  None          None      None     None       None    None      None
300   1981  None          None      None     None       None    None      None
417   1982  None          None      None     None       None    None      None
550   1983  None          None      None     None       None    None      None
710   1984  None          None      None     None       None    None      None
876   1985  None          None      None     None   

,Year,Rank,Release Group,Worldwide,Domestic,Domestic %,Foreign,Foreign %
1,1977,1,Star Wars: Episode IV - A New Hope,"$307,263,857","$307,263,857",100%,-,-
2,1977,2,Smokey and the Bandit,"$126,748,082","$126,737,428",100%,$87,<0.1%
3,1977,3,Close Encounters of the Third Kind,"$116,396,241","$116,395,460",100%,-,-
4,1977,4,Saturday Night Fever,"$94,214,336","$94,213,184",100%,-,-
5,1977,5,A Bridge Too Far,"$50,750,000","$50,750,000",100%,-,-
...,...,...,...,...,...,...,...,...
8892,2024,196,Io e te dobbiamo parlare,"$9,886,115",-,-,"$9,886,115",100%
8893,2024,197,Listy do M. Pozegnania i powroty,"$9,782,064",-,-,"$9,782,064",100%
8894,2024,198,Rafadan Tayfa: Kapadokya,"$9,514,264",-,-,"$9,514,264",100%
8895,2024,199,The Chosen: S4 Episodes 4-6,"$9,482,744","$9,482,744",100%,-,-


In [ ]:
# Elimino las columnas 'Domestic %' y 'Foreign %'
dfranking_cine_depurado = dfranking_cine_depurado.drop(columns=['Domestic %', 'Foreign %'])

In [ ]:
# Funcion para limpiar caracteres
def limpiar_caracteres(x):
    if isinstance(x, str):
        return x.replace('$', '').replace(',', '').replace('-', '0')
    return x
#Convierto las columnas a float
for col in ['Worldwide', 'Domestic','Foreign']:
    dfranking_cine_depurado[col] = dfranking_cine_depurado[col].apply(limpiar_caracteres).astype(float)
# Verifico Cambios
dfranking_cine_depurado.head(2)

,Year,Rank,Release Group,Worldwide,Domestic,Foreign
1,1977,1,Star Wars: Episode IV - A New Hope,307263857.0,307263857.0,0.0
2,1977,2,Smokey and the Bandit,126748082.0,126737428.0,87.0


In [ ]:
# Verifico que la depuracion fue efectiva
dfranking_cine_depurado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8849 entries, 1 to 8896
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           8849 non-null   int64  
 1   Rank           8849 non-null   object 
 2   Release Group  8849 non-null   object 
 3   Worldwide      8849 non-null   float64
 4   Domestic       8849 non-null   float64
 5   Foreign        8849 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 483.9+ KB


## Web Scrapping - Lanzamientos de las franquicias mas relevantes

Fuente de los datos: https://www.boxofficemojo.com/

Tarea desarrollada:
* Webscrapping de los lanzamientos de las franquicias mas reconocidas
* Transformacion y limpieza de datos
* Confeccion del dataset

In [ ]:
# Genero un diccionario de franquicias para el scrapping
listado_franquicias = {
    'Marvel Cinematic Universe': 'fr541495045',
    'Star Wars': 'fr3125251845',
    'Disney Live Action Reimaginings': 'fr1715965701',
    'Mission: Impossible': 'fr3678899973',
    'The Hunger Games': 'fr239570693',
    'Transformers': 'fr3863449349',
    'Pirates of the Caribbean': 'fr3494350597',
    'Star Trek': 'fr2806484741',
    'The Fast and the Furious': 'fr3628568325',
    'Jurassic Park': 'fr2571603717',
    'DC Extended Universe': 'fr340233989',
    'Batman': 'fr2286391045',
    'Spider-Man': 'fr3662122757',
    'X-Men': 'fr3712454405',
    'Superman': 'fr2756153093',
    'Bourne': 'fr3561459461',
    'Kung Fu Panda': 'fr206016261',
    'Sonic the Hedgehog': 'fr2605027077',
    'John Wick': 'fr1313312517',
    'The Matrix': 'fr3142029061',
    'Alien': 'fr2403831557',
    'Teenage Mutant Ninja Turtles': 'fr2722598661',
    'Men in Black': 'fr3846672133',
    'Monsterverse': 'fr289902341',
    'Rocky': 'fr2840039173',
    'The Lord of the Rings': 'fr2689044229',
    'The Hobbit': 'fr491228933',
    'Despicable Me': 'fr256347909',
    'James Bond': 'fr2605158149',
    'Terminator': 'fr3175583493',
    'Predator': 'fr3225915141',
    'Resident Evil': 'fr3578236677',
    'The Purge': 'fr424120069',
    'Mad Max': 'fr2504494853',
    'The Muppets': 'fr2420608773',
    'Taken': 'fr4689669',
    'Friday the 13th': 'fr2152173317',
    'Back to the Future': 'fr2319945477',
    'Lethal Weapon': 'fr2538049285',
    'Die Hard': 'fr2269613829',
    'Venom': 'fr2537918213',
    'LEGO': 'fr508006149',
    'Indiana Jones': 'fr3209137925',
    'Toy Story': 'fr3796340485',
    'Shrek': 'fr3746008837',
    'Avatar': 'fr877104901',
    'Joker':'fr2487586565',
    'Fantastic Four':'fr742887173',
    'Blade':'fr3729231621'
}

In [ ]:
franquicias_releases_list = []

for franchise_name, franchise_id in listado_franquicias.items():
    url_franchise = f"https://www.boxofficemojo.com/franchise/{franchise_id}/"

    try:
        response_franchise = requests.get(url_franchise)
        response_franchise.raise_for_status()

        soup_franchise = BeautifulSoup(response_franchise.text, 'html.parser')
        table_franchise = soup_franchise.find('table')

        if table_franchise:
            rows_franchise = table_franchise.find_all('tr')

            for row in rows_franchise:
                columns = row.find_all('td')
                if columns:
                    try:
                        # Extraer nombre y link del release
                        release_tag = columns[1].find('a')
                        release = release_tag.text.strip() if release_tag else columns[1].text.strip()
                        release_link = (
                            f"https://www.boxofficemojo.com{release_tag['href']}"
                            if release_tag and release_tag.has_attr('href') else None
                        )

                        release_date = columns[6].text.strip()
                        distributor = columns[7].text.strip()

                        franquicias_releases_list.append([
                            franchise_name, release, release_date, distributor, release_link
                        ])
                    except IndexError:
                        print(f"Salteado el registro incompleto: {franchise_name}")
                        continue
        else:
            print(f"No se encontró tabla para la franquicia: {franchise_name}")

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos de {franchise_name}: {e}")
        continue

# Crea el DataFrame incluyendo el link
franquicias_releases = pd.DataFrame(
    franquicias_releases_list,
    columns=['Franchise', 'Release', 'Release_Date', 'Distributor', 'BOM_Link']
)


In [ ]:
# Convierte 'Release_Date' a formato datetime
franquicias_releases['Release_Date'] = pd.to_datetime(franquicias_releases['Release_Date'], errors='coerce')

# Muestra para verificar los cambios
franquicias_releases.info()

# Display the head to see the format
franquicias_releases.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Franchise     508 non-null    object        
 1   Release       508 non-null    object        
 2   Release_Date  474 non-null    datetime64[ns]
 3   Distributor   508 non-null    object        
 4   BOM_Link      508 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 20.0+ KB


,Franchise,Release,Release_Date,Distributor,BOM_Link
0,Marvel Cinematic Universe,Avengers: Endgame,2019-04-26,Walt Disney Studios Motion Pictures,https://www.boxofficemojo.com/release/rl305997...
1,Marvel Cinematic Universe,Spider-Man: No Way Home,2021-12-17,Sony Pictures Releasing,https://www.boxofficemojo.com/release/rl286965...
2,Marvel Cinematic Universe,Black Panther,2018-02-16,Walt Disney Studios Motion Pictures,https://www.boxofficemojo.com/release/rl299286...
3,Marvel Cinematic Universe,Avengers: Infinity War,2018-04-27,Walt Disney Studios Motion Pictures,https://www.boxofficemojo.com/release/rl304319...
4,Marvel Cinematic Universe,Deadpool & Wolverine,2024-07-26,Walt Disney Studios Motion Pictures,https://www.boxofficemojo.com/release/rl410809...


## Web Scrapping - Recaudacion diaria en EEUU de los lanzamientos basados en comics desde el 2000 a la fecha

Fuente de los datos: https://www.boxofficemojo.com/

Tarea desarrollada:
* Webscrapping de la recaudación de los lanzamientos basados en comics
* Transformacion y limpieza de datos
* Confeccion del dataset

### Datos lanzamientos CBM

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/AI_Generated_Releases_Cinemas_Digital_Streaming.csv'

cbm_releases = pd.read_csv(file_path, encoding='latin-1')

In [ ]:
# Para evitar errores dejo solo el primer valor en caso de duplicados
franquicias_releases_unique = franquicias_releases.drop_duplicates(subset=['Release'], keep='first')

# Hago el Merge para sumarle el dato del link al DF BOM_Link
cbm_releases = pd.merge(cbm_releases, franquicias_releases_unique[['Release', 'BOM_Link']], on='Release', how='left')

cbm_releases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Release                    89 non-null     object
 1   Studio                     89 non-null     object
 2   Release_Cinema             89 non-null     object
 3   Release_Digital_Streaming  89 non-null     object
 4   BOM_Link                   82 non-null     object
dtypes: object(5)
memory usage: 3.6+ KB


#### Limpieza, correccion y complementacion de datos.

In [ ]:
cbm_releases = cbm_releases.drop(84)
cbm_releases = cbm_releases.drop(88)

In [ ]:
faltantes = {79:'https://www.boxofficemojo.com/release/rl2403829249/',82:"https://www.boxofficemojo.com/release/rl3662710273/",45:"https://www.boxofficemojo.com/release/rl3864036865/",37:"https://www.boxofficemojo.com/release/rl2064025089/",38:"https://www.boxofficemojo.com/release/rl2812183041/",39:"https://www.boxofficemojo.com/release/rl1379697409/",53:"https://www.boxofficemojo.com/title/tt0409459/",54:"https://www.boxofficemojo.com/release/rl1280804353/",73:"https://www.boxofficemojo.com/release/rl2339995649/"}
for index, link in faltantes.items():
    cbm_releases.iloc[index, 4] = link
cbm_releases.iloc[73,0]="Joker: Folie à Deux"


In [ ]:
# Verifico que no haya registros con datos faltantes
null_bom_links = cbm_releases[cbm_releases['BOM_Link'].isnull()]
display(null_bom_links)

,Release,Studio,Release_Cinema,Release_Digital_Streaming,BOM_Link


In [ ]:
cols_to_datetime =["Release_Cinema","Release_Digital_Streaming"]
cbm_releases[cols_to_datetime] = cbm_releases[cols_to_datetime].apply(pd.to_datetime, format='%d/%m/%Y', errors='coerce')
cbm_releases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 0 to 87
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Release                    87 non-null     object        
 1   Studio                     87 non-null     object        
 2   Release_Cinema             87 non-null     datetime64[ns]
 3   Release_Digital_Streaming  87 non-null     datetime64[ns]
 4   BOM_Link                   87 non-null     object        
dtypes: datetime64[ns](2), object(3)
memory usage: 4.1+ KB


In [ ]:
display(cbm_releases[cbm_releases['Release']=="Venom: The Last Dance"])

,Release,Studio,Release_Cinema,Release_Digital_Streaming,BOM_Link
40,Venom: The Last Dance,Sony,2024-10-25,2024-12-10,https://www.boxofficemojo.com/release/rl329000...


### Scrapping recaudaciones diarias

In [ ]:
recaudacion_diaria_cbm = []

# Itera a traves de cada fila del dataframe cbm_releases
for index, row in cbm_releases.iterrows():
    release = row['Release']
    studio = row['Studio']
    release_cinema = row['Release_Cinema']
    release_digital_streaming = row['Release_Digital_Streaming']
    url_release = row['BOM_Link']

    if url_release:
        try:
            # Realiza la solicitud HTTP
            response_release = requests.get(url_release)
            response_release.raise_for_status() # Lanza un error para respuestas HTTP incorrectas

            # Analiza el contenido HTML
            soup_release = BeautifulSoup(response_release.text, 'html.parser')

            # Busca la tabla de recaudacion diaria
            table_release = soup_release.find('table')

            if table_release:
                # Extrae los datos de la tabla
                rows_release = table_release.find_all('tr')

                # Procesa los datos de la tabla, agregando la informacion del release
                for row_data in rows_release:
                    columns = row_data.find_all('td')
                    if columns: # Asegura de que la fila no este vacia
                        data = [col.text.strip() for col in columns]
                        # Añade la informacion del release al inicio de cada fila de datos diarios
                        data.insert(0, release_digital_streaming)
                        data.insert(0, release_cinema)
                        data.insert(0, studio)
                        data.insert(0, release)
                        recaudacion_diaria_cbm.append(data)
            else:
                print(f"No se encontró la tabla de recaudación diaria para: {release} ({url_release})")

        except requests.exceptions.RequestException as e:
            print(f"Error al obtener datos de {release} ({url_release}): {e}")
        except Exception as e:
            print(f"Error procesando datos de {release} ({url_release}): {e}")
    else:
        print(f"Link de BOM faltante para: {release}")



In [ ]:
# Genero los nombres de las columnas
recaudacion_diaria_cbm_cols = ['Release', 'Studio', 'Release_Cinema', 'Release_Digital_Streaming','Date',"DOW","Rank","Daily",	"%± YD","%± LW","Theaters","Avg",	"To Date",	"Day","1"]
# Genero el dataset con los datos scrapeados y los nombre de las columnas
dfrecaudacion_diaria_cbm = pd.DataFrame(recaudacion_diaria_cbm, columns = recaudacion_diaria_cbm_cols)

In [ ]:
# Correccion de valor
dfrecaudacion_diaria_cbm['Release_Digital_Streaming'].fillna(pd.to_datetime('2024-12-10'), inplace=True)

/tmp/ipython-input-1897321150.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfrecaudacion_diaria_cbm['Release_Digital_Streaming'].fillna(pd.to_datetime('2024-12-10'), inplace=True)


In [ ]:
# Limpia y convierte 'Theaters' a int
dfrecaudacion_diaria_cbm['Theaters'] = dfrecaudacion_diaria_cbm['Theaters'].astype(str).str.replace(',', '', regex=False)
dfrecaudacion_diaria_cbm['Theaters'] = pd.to_numeric(dfrecaudacion_diaria_cbm['Theaters'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Convierte 'Day' y 'Rank' a int, manejando valores no numéricos
dfrecaudacion_diaria_cbm['Day'] = pd.to_numeric(dfrecaudacion_diaria_cbm['Day'], errors='coerce').fillna(0).astype(int)
dfrecaudacion_diaria_cbm['Rank'] = pd.to_numeric(dfrecaudacion_diaria_cbm['Rank'], errors='coerce').fillna(0).astype(int)
# Limpia y convierte 'Theaters' a int
dfrecaudacion_diaria_cbm['Theaters'] = dfrecaudacion_diaria_cbm['Theaters'].astype(str).str.replace(',', '', regex=False)
dfrecaudacion_diaria_cbm['Theaters'] = pd.to_numeric(dfrecaudacion_diaria_cbm['Theaters'], errors='coerce').fillna(0).astype(int)
# Limpia y converte 'Daily' a float
dfrecaudacion_diaria_cbm['Daily'] = dfrecaudacion_diaria_cbm['Daily'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
dfrecaudacion_diaria_cbm['Daily'] = pd.to_numeric(dfrecaudacion_diaria_cbm['Daily'], errors='coerce').fillna(0)

dfrecaudacion_diaria_cbm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8810 entries, 0 to 8809
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Release                    8810 non-null   object        
 1   Studio                     8810 non-null   object        
 2   Release_Cinema             8810 non-null   datetime64[ns]
 3   Release_Digital_Streaming  8810 non-null   datetime64[ns]
 4   Date                       8810 non-null   object        
 5   DOW                        8810 non-null   object        
 6   Rank                       8810 non-null   int64         
 7   Daily                      8810 non-null   float64       
 8   %± YD                      8809 non-null   object        
 9   %± LW                      8809 non-null   object        
 10  Theaters                   8810 non-null   int64         
 11  Avg                        8809 non-null   object        
 12  To Dat

In [ ]:
cols_del = ['1', 'To Date', 'Avg', '%± LW','%± YD', 'Date']
dfrecaudacion_diaria_cbm.drop(columns=cols_del, inplace=True)
dfrecaudacion_diaria_cbm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8810 entries, 0 to 8809
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Release                    8810 non-null   object        
 1   Studio                     8810 non-null   object        
 2   Release_Cinema             8810 non-null   datetime64[ns]
 3   Release_Digital_Streaming  8810 non-null   datetime64[ns]
 4   DOW                        8810 non-null   object        
 5   Rank                       8810 non-null   int64         
 6   Daily                      8810 non-null   float64       
 7   Theaters                   8810 non-null   int64         
 8   Day                        8810 non-null   int64         
dtypes: datetime64[ns](2), float64(1), int64(3), object(3)
memory usage: 619.6+ KB


In [ ]:
dfrecaudacion_diaria_cbm

,Release,Studio,Release_Cinema,Release_Digital_Streaming,DOW,Rank,Daily,Theaters,Day
0,Iron Man,MCU,2008-05-02,2008-09-30,Thursday,1,3500000.0,2500,0
1,Iron Man,MCU,2008-05-02,2008-09-30,Friday,1,35234361.0,4105,1
2,Iron Man,MCU,2008-05-02,2008-09-30,Saturday,1,37350099.0,4105,2
3,Iron Man,MCU,2008-05-02,2008-09-30,Sunday,1,26034208.0,4105,3
4,Iron Man,MCU,2008-05-02,2008-09-30,Monday,1,6934568.0,4105,4
...,...,...,...,...,...,...,...,...,...
8805,X-Men: The Last Stand,Fox,2006-05-26,2006-10-03,Sunday,43,4398.0,39,108
8806,X-Men: The Last Stand,Fox,2006-05-26,2006-10-03,Monday,40,1571.0,39,109
8807,X-Men: The Last Stand,Fox,2006-05-26,2006-10-03,Tuesday,39,2518.0,39,110
8808,X-Men: The Last Stand,Fox,2006-05-26,2006-10-03,Wednesday,40,1649.0,39,111


## Web Scrapping - Recaudacion diaria de los 10 lanzamientos mas importantes desde 1989 a la fecha

Fuente de los datos: https://www.boxofficemojo.com/

Tarea:
* Webscrapping de la recaudacion diariadesde el año 1989 a la actualidad
* Transformacion y limpieza de datos
* Confeccion del dataset

In [ ]:
recaudacion_diaria = []
año = datetime.datetime.now().year
# Scrapea los datos desde 1989 hasta el año inmediato anterior a la consulta (2024 a fecha de subida)
for i in range (1989,año):
    url_año = f"https://www.boxofficemojo.com/daily/{i}/"

    # Realizar la solicitud HTTP
    response_año = requests.get(url_año)

    # Analizar el contenido HTML
    soup_año = BeautifulSoup(response_año.text, 'html.parser')

    # Buscar la tabla de debilidades
    table_año = soup_año.find('table')

    if table_año: # Add a check to see if the table was found
        # Extraer los datos de la tabla
        rows_año = table_año.find_all('tr')

        # Procesar los datos de la tabla

        for row in rows_año:
            columns = row.find_all('td')
            data = [col.text for col in columns]
            data.insert(0,i)
            recaudacion_diaria.append(data)
    else:
        print(f"No se encontro la tabla para el año {i}") # Imprime el mensaje si no encuentra la tabla

# Genero los nombres de las columnas
recaudacion_diaria_cols = ["Year","Date","Day","Day#","Top_10_Gross","%±YD","%±LW","Releases","#1_Release","#1_Gross"]
# Genero el dataset con los datos scrapeados y los nombre de las columnas
dfrecaudacion_diaria = pd.DataFrame(recaudacion_diaria, columns = recaudacion_diaria_cols)

dfrecaudacion_diaria.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12811 entries, 0 to 12810
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Year          12811 non-null  int64 
 1   Date          12775 non-null  object
 2   Day           12775 non-null  object
 3   Day#          12775 non-null  object
 4   Top_10_Gross  12775 non-null  object
 5   %±YD          12775 non-null  object
 6   %±LW          12775 non-null  object
 7   Releases      12775 non-null  object
 8   #1_Release    12775 non-null  object
 9   #1_Gross      12775 non-null  object
dtypes: int64(1), object(9)
memory usage: 1001.0+ KB


In [ ]:
# El primer registro de cada año esta vacio, se elimina
dfrecaudacion_diaria.dropna(inplace=True)
dfrecaudacion_diaria.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12775 entries, 1 to 12810
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Year          12775 non-null  int64 
 1   Date          12775 non-null  object
 2   Day           12775 non-null  object
 3   Day#          12775 non-null  object
 4   Top_10_Gross  12775 non-null  object
 5   %±YD          12775 non-null  object
 6   %±LW          12775 non-null  object
 7   Releases      12775 non-null  object
 8   #1_Release    12775 non-null  object
 9   #1_Gross      12775 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.1+ MB


In [ ]:
# Elimino columnas innecesarias
del_cols =['%±YD','%±LW']
dfrecaudacion_diaria = dfrecaudacion_diaria.drop(columns=del_cols)

In [ ]:
# Corrijo formatos
cols_to_float = ['Top_10_Gross','#1_Gross']
dfrecaudacion_diaria[cols_to_float] = dfrecaudacion_diaria[cols_to_float].apply(lambda x: x.str.replace('$', '').str.replace(',', '').astype(float))
cols_to_int = ['Day#','Releases']
dfrecaudacion_diaria[cols_to_int] = dfrecaudacion_diaria[cols_to_int].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Genero una columna con la feca normalizada
dfrecaudacion_diaria["Date_Normalized"] = pd.to_datetime(dfrecaudacion_diaria["Year"].astype(str), format='%Y') + pd.to_timedelta(dfrecaudacion_diaria["Day#"] - 1, unit='D')

In [ ]:
# Funcion para identificar el tipo de dia
# La columna Date tiene informacion del mes y dia pero en algunos caso si es un feriado o evento, aislo ese dato en una nueva columna
# Para el resto identifico si es fin de semana o dia normal (se toma el criterio de la pagina de considerar al viernes como parte del fin de semana)

def clasificar_dia(row):
    date = row["Date"]
    day = row["Day"]

    # Extrae texto que aparece luego del último dígito (si existe)
    match = re.search(r'\d+(.*)', date)
    if match and match.group(1).strip():  # si hay algo después del número
        return match.group(1).strip()  # es una denominación especial

    if day in ["Friday", "Saturday", "Sunday"]:
        return "Weekend"

    return "Normal"

# Aplica la función fila por fila
dfrecaudacion_diaria["Day_Type"] = dfrecaudacion_diaria.apply(clasificar_dia, axis=1)

In [ ]:
# Listo eventos individualizados
dfrecaudacion_diaria["Day_Type"].unique()

array(["New Year's Eve", 'Weekend', 'Normal', 'Christmas Day',
       'Thanksgiving', 'Halloween', "Indig. Peoples' Day", 'Labor Day',
       'Independence Day', 'Memorial Day', 'Easter Sunday',
       "Presidents' Day", 'MLK Day', "New Year's Day", 'World Cup (USA)',
       'Independence DayWorld Cup (USA)', 'Lillehammer Olympics',
       "Lillehammer OlympicsPresidents' Day", 'Atlanta Olympics',
       'World Cup (France)', 'Independence DayWorld Cup (France)',
       'Nagano Olympics', "Nagano OlympicsPresidents' Day",
       'Sydney Olympics', '/11 Attacks', 'World Cup (Korea/Japan)',
       'Salt Lake Olympics', "Presidents' DaySalt Lake Olympics",
       'Athens Olympics', 'Hurricane Katrina',
       'Hurricane KatrinaLabor Day', 'World Cup (Germany)',
       'Independence DayWorld Cup (Germany)', 'Torino Olympics',
       "Presidents' DayTorino Olympics", 'Beijing Olympics',
       'World Cup (South Africa)',
       'Independence DayWorld Cup (South Africa)', 'Vancouver Olympics

In [ ]:
# Mapeo para normalizar nombres de eventos
event_map = {
    'World Cup (USA)': 'World Cup',
    'World Cup (France)': 'World Cup',
    'World Cup (Korea/Japan)': 'World Cup',
    'World Cup (Germany)': 'World Cup',
    'World Cup (South Africa)': 'World Cup',
    'World Cup (Brazil)': 'World Cup',
    'World Cup (Russia)': 'World Cup',
    'World Cup (Qatar)': 'World Cup',

    'Lillehammer Olympics': 'Olympics',
    'Nagano Olympics': 'Olympics',
    'Sydney Olympics': 'Olympics',
    'Salt Lake Olympics': 'Olympics',
    'Athens Olympics': 'Olympics',
    'Torino Olympics': 'Olympics',
    'Beijing Olympics': 'Olympics',
    'Vancouver Olympics': 'Olympics',
    'London Olympics': 'Olympics',
    'Sochi Olympics': 'Olympics',
    'Rio Olympics': 'Olympics',
    'PyeongChang Olympics': 'Olympics',

    'Hurricane Katrina': 'Hurricane',
    'Hurricane Sandy': 'Hurricane',

    'COVID-19 Pandemic': 'COVID-19 Pandemic',
    "New Year's Eve": "New Year's Eve",
    "New Year's Day": "New Year's Day",
    'Thanksgiving': 'Thanksgiving',
    'Halloween': 'Halloween',
    "Indig. Peoples' Day": "Indig. Peoples' Day",
    'Labor Day': 'Labor Day',
    'Independence Day': 'Independence Day',
    'Memorial Day': 'Memorial Day',
    'Easter Sunday': 'Easter Sunday',
    "Presidents' Day": "Presidents' Day",
    'MLK Day': 'MLK Day',
    '/11 Attacks': '9/11 Attacks',
    'Mayweather/Pacquiao Fight': 'Mayweather/Pacquiao Fight',
    'Weekend': 'Weekend',
    'Normal': 'Normal'
}

# Función para descomponer y normalizar
def extract_events(date_type):
    found_events = []
    for key, norm in event_map.items():
        if key in date_type:
            found_events.append(norm)
    if found_events:
        event_princ = found_events[0]
        event_sec = found_events[1] if len(found_events) > 1 else ''
    else:
        event_princ = ''
        event_sec = ''
    return pd.Series([event_princ, event_sec])

# Aplica al DataFrame
dfrecaudacion_diaria[['Event_Primary', 'Event_Secondary']] = dfrecaudacion_diaria['Day_Type'].apply(extract_events)

In [ ]:
# Listo eventos individualizados
dfrecaudacion_diaria["Event_Primary"].unique(),dfrecaudacion_diaria["Event_Secondary"].unique()

(array(["New Year's Eve", 'Weekend', 'Normal', '', 'Thanksgiving',
        'Halloween', "Indig. Peoples' Day", 'Labor Day',
        'Independence Day', 'Memorial Day', 'Easter Sunday',
        "Presidents' Day", 'MLK Day', "New Year's Day", 'World Cup',
        'Olympics', '9/11 Attacks', 'Hurricane',
        'Mayweather/Pacquiao Fight', 'COVID-19 Pandemic'], dtype=object),
 array(['', 'Independence Day', "Presidents' Day", 'Labor Day',
        'Halloween', "New Year's Eve", 'Thanksgiving',
        "Indig. Peoples' Day", 'Memorial Day', 'Easter Sunday', 'MLK Day',
        "New Year's Day"], dtype=object))

In [ ]:
# Elimino la columna al ser desglozada
cols_del = ['Date','Day_Type']
dfrecaudacion_diaria.drop(columns=cols_del, inplace=True)

In [ ]:
# Verifico que los tipos de datos sean correctos
dfrecaudacion_diaria.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12775 entries, 1 to 12810
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Year             12775 non-null  int64         
 1   Day              12775 non-null  object        
 2   Day#             12775 non-null  int64         
 3   Top_10_Gross     12775 non-null  float64       
 4   Releases         12775 non-null  int64         
 5   #1_Release       12775 non-null  object        
 6   #1_Gross         12775 non-null  float64       
 7   Date_Normalized  12775 non-null  datetime64[ns]
 8   Event_Primary    12775 non-null  object        
 9   Event_Secondary  12775 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 1.1+ MB


In [ ]:
dfrecaudacion_diaria

,Year,Day,Day#,Top_10_Gross,Releases,#1_Release,#1_Gross,Date_Normalized,Event_Primary,Event_Secondary
1,1989,Sunday,365,1715715.0,2,Back to the Future Part II,1675065.0,1989-12-31,New Year's Eve,
2,1989,Saturday,364,2586665.0,2,Back to the Future Part II,2528400.0,1989-12-30,Weekend,
3,1989,Friday,363,2342700.0,2,Back to the Future Part II,2296630.0,1989-12-29,Weekend,
4,1989,Thursday,362,1969765.0,2,Back to the Future Part II,1965370.0,1989-12-28,Normal,
5,1989,Wednesday,361,1833135.0,2,Back to the Future Part II,1828740.0,1989-12-27,Normal,
...,...,...,...,...,...,...,...,...,...,...
12806,2024,Friday,5,24699285.0,46,Night Swim,5246275.0,2024-01-05,Weekend,
12807,2024,Thursday,4,12533498.0,46,Wonka,2703155.0,2024-01-04,Normal,
12808,2024,Wednesday,3,14266846.0,45,Wonka,3152613.0,2024-01-03,Normal,
12809,2024,Tuesday,2,20006635.0,45,Wonka,4601670.0,2024-01-02,Normal,


## Web Scrapping - Recaudacion feriados desde 1989 a la fecha

Fuente de los datos: https://www.boxofficemojo.com/

Tarea:
* Webscrapping de la recaudacion diariadesde el año 1989 a la actualidad
* Transformacion y limpieza de datos
* Confeccion del dataset

In [ ]:
#https://www.boxofficemojo.com/holiday/by-year/2024/

recaudacion_feriados = []
año = datetime.datetime.now().year
# Scrapea los datos desde 1989 hasta el año inmediato anterior a la consulta (2024 a fecha de subida)
for i in range (1989,año):
    url_año = f"https://www.boxofficemojo.com/holiday/by-year/{i}/"

    # Realizar la solicitud HTTP
    response_año = requests.get(url_año)

    # Analizar el contenido HTML
    soup_año = BeautifulSoup(response_año.text, 'html.parser')

    # Buscar la tabla de debilidades
    table_año = soup_año.find('table')

    if table_año: # Add a check to see if the table was found
        # Extraer los datos de la tabla
        rows_año = table_año.find_all('tr')

        # Procesar los datos de la tabla

        for row in rows_año:
            columns = row.find_all('td')
            data = [col.text for col in columns]
            data.insert(0,i)
            recaudacion_feriados.append(data)
    else:
        print(f"No se encontro la tabla para el año {i}") # Imprime el mensaje si no encuentra la tabla

recaudacion_feriados_cols = ["Year","Cumulative Gross","%_of_Year","Releases","Average","#1_Release","Gross_#1","%_of_Total"]


In [ ]:
recaudacion_feriados_cols = ["Year","Holiday","Cumulative Gross","%_of_Year","Releases","Average","#1_Release","1","2","3","#1_Gross","%_of_Total"]
dfrecaudacion_feriados = pd.DataFrame(recaudacion_feriados, columns = recaudacion_feriados_cols)

In [ ]:
cols_to_del = ['1','2','3',"%_of_Year","%_of_Total","Average"]
dfrecaudacion_feriados.drop(columns=cols_to_del, inplace=True)


In [ ]:
cols_to_int = ["Releases"]
dfrecaudacion_feriados[cols_to_int] = dfrecaudacion_feriados[cols_to_int].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Elimina registros con valores null
dfrecaudacion_feriados.dropna(subset=['Holiday'], inplace=True)

In [ ]:
# Funcion para limpiar caracteres
def limpiar_caracteres(x):
    if isinstance(x, str):
        return x.replace('$', '').replace(',', '').replace('-', '0')
    return x
#Convierto las columnas a float
for col in ['Cumulative Gross', '#1_Gross']:
    dfrecaudacion_feriados[col] = dfrecaudacion_feriados[col].apply(limpiar_caracteres).astype(float)

In [ ]:
# Borrar valores null
# reemplazar por 0 los Releases faltantes
dfrecaudacion_feriados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 836 entries, 1 to 871
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              836 non-null    int64  
 1   Holiday           836 non-null    object 
 2   Cumulative Gross  836 non-null    float64
 3   Releases          824 non-null    float64
 4   #1_Release        836 non-null    object 
 5   #1_Gross          836 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 45.7+ KB


In [ ]:
dfrecaudacion_feriados

,Year,Holiday,Cumulative Gross,Releases,#1_Release,#1_Gross
1,1989,New Year's Eve,1715715.0,2.0,Back to the Future Part II,1675065.0
2,1989,Christmas Day,898190.0,2.0,Back to the Future Part II,851310.0
3,1989,Christmas Weekend,45702925.0,24.0,National Lampoon's Christmas Vacation,7007789.0
4,1989,Post-Thanksgiving Weekend,49740789.0,14.0,Back to the Future Part II,12110340.0
5,1989,Thanksgiving Weekend,79985363.0,14.0,Back to the Future Part II,27835125.0
...,...,...,...,...,...,...
867,2024,President's Day,15301412.0,43.0,Bob Marley: One Love,4974818.0
868,2024,President's Day Weekend,94237683.0,51.0,Bob Marley: One Love,33633822.0
869,2024,MLK Day,19947590.0,44.0,Mean Girls,4975603.0
870,2024,MLK Day Weekend,116912802.0,52.0,Mean Girls,33610910.0


## Scapping - Precio promedio de las entradas de cine en EEUU

### PDF Scrapping - 1948 a 2017

Fuente de los datos: https://illinoistreasurergovprod.blob.core.usgovcloudapi.net/twocms/media/doc/6-8.1.1%20movie%20ticket%20prices.pdf

Tarea desarrollada:
* PDF scrapping de la recaudación de los lanzamientos basados en comics
* Transformacion de datos
  

In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 79.9 MB/s eta 0:00:00


In [ ]:
import pdfplumber

In [ ]:
# Ruta al PDF (ya subido en tu Colab)
pdf_path = "/content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/6-8.1.1 movie ticket prices.pdf"

years = []
prices = []

# Convierto el pdf a una cadena
with pdfplumber.open(pdf_path) as pdf:
    full_text = ""
    for page in pdf.pages:
        full_text += page.extract_text() + "\n"

# Regex para capturar año y precio
matches = re.findall(r"(\d{4})\s+\$?(\d+\.\d+)", full_text)

# Crea DataFrame
entradas1948_2017 = pd.DataFrame(matches, columns=["Year", "Price"])
entradas1948_2017["Year"] = entradas1948_2017["Year"].astype(int)
entradas1948_2017["Price"] = entradas1948_2017["Price"].astype(float)

# Ordena por año
entradas1948_2017 = entradas1948_2017.sort_values("Year").reset_index(drop=True)


In [ ]:
entradas1948_2017.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    50 non-null     int64  
 1   Price   50 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 932.0 bytes


### Web Scrapping - 1995 a 2025

Fuente de los datos: https://www.the-numbers.com/market/

Tarea:
* Webscrapping del precio promedio de las entradas desde 1995 a 2025
* Correccion de formatos

En esta oportunidad se opta por **pd.read_html(url)**, una función de pandas que:

* Descarga el contenido HTML de la URL (o archivo local .html).
* Busca todas las tablasdentro del HTML.
* Convierte cada tabla encontrada en un DataFrame de pandas.
* Devuelve una lista de DataFrames, uno por cada tabla detectada.

In [ ]:
# URL de la página
url = "https://www.the-numbers.com/market/"

# Leo todas las tablas de la página
tables = pd.read_html(url)

In [ ]:
# Selecciono la primera tabla (asumido que es la que necesitas)
entradas1995_2025 = tables[0]

# Listo las primeras filas para confirmar columnas
print(entradas1995_2025.head())

   Year  Tickets Sold Total Box Office Total Inflation Adjusted Box Office  \
0  2025     763169386   $8,631,445,757                      $8,631,445,757   
1  2024     760482781   $8,601,060,254                      $8,601,060,254   
2  2023     819306691   $8,963,217,610                      $9,266,358,669   
3  2022     702373460   $7,395,994,572                      $7,943,843,838   
4  2021     444250543   $4,518,029,580                      $5,024,473,637   

  Average Ticket Price  
0               $11.31  
1               $11.31  
2               $10.94  
3               $10.53  
4               $10.17  


In [ ]:
# Convierto 'Year' a entero
entradas1995_2025['Year'] = entradas1995_2025['Year'].astype(int)

# Columnas numéricas excepto 'Year'
num_cols = [col for col in entradas1995_2025.columns if col != 'Year']

# Limpio y convierto a float
for col in num_cols:
    entradas1995_2025[col] = (
        entradas1995_2025[col]
          .astype(str)
          .str.replace(r'[\$,]', '', regex=True)  # quitar $ y comas
          .astype(float)
    )

# Ordeno por año (opcional)
entradas1995_2025 = entradas1995_2025.sort_values('Year').reset_index(drop=True)

# Chequeo datos
entradas1995_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 31 non-null     int64  
 1   Tickets Sold                         31 non-null     float64
 2   Total Box Office                     31 non-null     float64
 3   Total Inflation Adjusted Box Office  31 non-null     float64
 4   Average Ticket Price                 31 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 1.3 KB


### Precio Promedio entradas EEUU 1995 a 2025 - Unificado

* Confeccion del dataframe

In [ ]:
# Filtro el primer DataFrame entre 1977 y 2017
df1 = entradas1948_2017[(entradas1948_2017["Year"] >= 1977) & (entradas1948_2017["Year"] <= 2017)][["Year", "Price"]]

# Filtro el segundo DataFrame entre 2018 y 2025
df2 = entradas1995_2025[(entradas1995_2025["Year"] >= 2018) & (entradas1995_2025["Year"] <= 2025)][["Year", "Average Ticket Price"]]

# Renombro la columna para que coincidan
df2 = df2.rename(columns={"Average Ticket Price": "Price"})

# Unifico ambos DataFrames en el rango de años deseado
dfprecio_entradas1977_2025 = pd.concat([df1, df2]).sort_values("Year").reset_index(drop=True)

In [ ]:
dfprecio_entradas1977_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    49 non-null     int64  
 1   Price   49 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 916.0 bytes


## Web Scrapping - Cantidad de tickes vendidos y recaudacion total EEUU 1995 a 2025

Fuente de los datos: https://www.the-numbers.com/market/

Tarea:
* Webscrapping de la cantidad de tickets vendidos desde 1995 a 2025
* Transformacion


In [ ]:
# Genero el Dataframe con los datos del scrapping
dfticketsvendidos1995_2025 =tables[0]

In [ ]:
# Convierto a int la cantidad de tickets
dfticketsvendidos1995_2025 ['Tickets Sold'] = dfticketsvendidos1995_2025 ['Tickets Sold'].astype(int)
dfticketsvendidos1995_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 31 non-null     int64  
 1   Tickets Sold                         31 non-null     int64  
 2   Total Box Office                     31 non-null     float64
 3   Total Inflation Adjusted Box Office  31 non-null     float64
 4   Average Ticket Price                 31 non-null     float64
dtypes: float64(3), int64(2)
memory usage: 1.3 KB


## Web Scarpping - Cantidad lanzamientos por estudio desde 1995 a 2025

Fuente de los datos: https://www.the-numbers.com/market/

Tarea:
* Webscrapping de la cantidad de lanzamientos por estudio desde 1995 a 2025
* Transformacion


In [ ]:
dflanzamientos_por_estudio1995_2025 = tables[1]

In [ ]:
columnas = list[dflanzamientos_por_estudio1995_2025.columns]

for columna in columnas:
    dflanzamientos_por_estudio1995_2025[columna] = dflanzamientos_por_estudio1995_2025[columna].astype(int)

In [ ]:
dflanzamientos_por_estudio1995_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Year                 31 non-null     int64
 1   Warner Bros          31 non-null     int64
 2   Walt Disney          31 non-null     int64
 3   20th Century Fox     31 non-null     int64
 4   Paramount Pictures   31 non-null     int64
 5   Sony Pictures        31 non-null     int64
 6   Universal            31 non-null     int64
 7   Total Major 6        31 non-null     int64
 8   Total Other Studios  31 non-null     int64
dtypes: int64(9)
memory usage: 2.3 KB


----
## Web Scrapping - Indices de precios EEUU




Fuente de los datos: https://www.bcentral.cl/inicio

Tarea desarrollada:
* Transformacion y limpieza de datos
* Confeccion del dataset

### Series anuales

In [ ]:
# Genero la cnsulta con un f string para que se adapte al momento que se la haga
consulta_ind= f'https://si3.bcentral.cl/Siete/ES/Siete/Cuadro/CAP_EI/MN_EI11/EI_IndicePrecios_EEUU/637297579678797084?cbFechaInicio=1977&cbFechaTermino={año-1}&cbFrecuencia=ANNUAL&cbCalculo=NONE&cbFechaBase='
url_ind = consulta_ind

# Realizo la solicitud HTTP
response_ind = requests.get(url_ind)

# Analizo el contenido HTML
soup_ind = BeautifulSoup(response_ind.text, 'html.parser')

# Busco la tabla de debilidades
table_ind = soup_ind.find('table')

# Extraigo los datos de la tabla
rows_ind = table_ind.find_all('tr')

# Procesa los datos de la tabla
indices_inflacion = []
for row in rows_ind:
    columns = row.find_all('td')
    data = [col.text.replace('\n', '') for col in columns]
    indices_inflacion.append(data)

In [ ]:
# Genero el nombre de las columnas del dataframe
indices_cols = ['X','Serie']
for i in range(1977,año):
    indices_cols.append(i)
# Gereno el dataframe con la lista de indices mas los nombres de las columnas
df_indices = pd.DataFrame(indices_inflacion, columns = indices_cols)
# Borro una columna con datos NULL
df_indices.drop(columns=['X'], inplace=True)
# Borro el primer registroi que es NULL
df_indices = df_indices.drop(0)
# Transpongo el dataframe
df_indices = df_indices.transpose()
df_indices = df_indices.rename(columns=df_indices.iloc[0]).drop(df_indices.index[0])

In [ ]:
df_indices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 1977 to 2024
Data columns (total 4 columns):
 #   Column                                                                   Non-Null Count  Dtype 
---  ------                                                                   --------------  ----- 
 0   IPC Todos los ítemes (promedio 1982-1984=100)                            48 non-null     object
 1   IPC Todos los ítemes menos alimentos y energía (promedio 1982-1984=100)  48 non-null     object
 2   IPP Bienes terminados (promedio 1982=100)                                48 non-null     object
 3   IPP  Todos los commodities (promedio 1982=100)                           48 non-null     object
dtypes: object(4)
memory usage: 1.9+ KB


In [ ]:
# Convierto estas columnas a Float
lista_conv = ['IPC Todos los ítemes (promedio 1982-1984=100)',
              'IPC Todos los ítemes menos alimentos y energía (promedio 1982-1984=100)',
              'IPP Bienes terminados (promedio 1982=100)',
              'IPP  Todos los commodities (promedio 1982=100)']
# Reemplazo , por .
df_indices[lista_conv] = df_indices[lista_conv].apply(lambda x: x.str.replace(',', '.') if x.dtype == "object" else x)
# Convierto a dato numerico
df_indices[lista_conv] = df_indices[lista_conv].apply(pd.to_numeric, errors='coerce')
# Verifico el cambio
df_indices.info(),df_indices.loc[2024]

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 1977 to 2024
Data columns (total 4 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   IPC Todos los ítemes (promedio 1982-1984=100)                            48 non-null     float64
 1   IPC Todos los ítemes menos alimentos y energía (promedio 1982-1984=100)  48 non-null     float64
 2   IPP Bienes terminados (promedio 1982=100)                                48 non-null     float64
 3   IPP  Todos los commodities (promedio 1982=100)                           48 non-null     float64
dtypes: float64(4)
memory usage: 1.9+ KB


(None,
 IPC Todos los ítemes (promedio 1982-1984=100)                              313.7
 IPC Todos los ítemes menos alimentos y energía (promedio 1982-1984=100)    319.0
 IPP Bienes terminados (promedio 1982=100)                                  257.7
 IPP  Todos los commodities (promedio 1982=100)                             254.6
 Name: 2024, dtype: float64)

In [ ]:
# Genero la columna year en base al indice del DF
df_indices = df_indices.reset_index()
df_indices = df_indices.rename(columns={'index': 'year'})
# Convierto el dato a numerico
df_indices['year'] = pd.to_numeric(df_indices['year'], errors='coerce')
# Muestro los cambios
df_indices.head()

,year,IPC Todos los ítemes (promedio 1982-1984=100),IPC Todos los ítemes menos alimentos y energía (promedio 1982-1984=100),IPP Bienes terminados (promedio 1982=100),IPP Todos los commodities (promedio 1982=100)
0,1977,60.6,61.0,64.7,64.9
1,1978,65.2,65.5,69.8,69.9
2,1979,72.6,71.9,77.6,78.7
3,1980,82.4,80.8,88.0,89.8
4,1981,90.9,89.2,96.2,98.0


---
# Guardado de los DataFrames en Google Drive


In [ ]:
directory_path = input("Indicar la ruta de Google Drive para almacenar el archivo: ")
df_to_save = {"dfranking_cine_depurado":"scrapping_RankingCine200.csv",
              "franquicias_releases":"scrapping_FranquiciasReleases.csv",
              "dfrecaudacion_diaria_cbm":"scrapping_CBM_RecaudacionDiaria.csv",
              "dfrecaudacion_diaria":"scrapping_RecaudacionDiaria10.csv",
              "dfrecaudacion_feriados":"scrapping_RecaudacionFeriados.csv",
              "cbm_releases":"scrapping_CBM_Releases.csv",
              "dfprecio_entradas1977_2025":"scrapping_PrecioEntradas1977_2025.csv",
              "dfticketsvendidos1995_2025":"scrapping_TicketsVendidos1995_2025.csv",
              "dflanzamientos_por_estudio1995_2025":"scrapping_LanzamientosPorEstudio1995_2025.csv",
              "df_indices":"scrapping_IndicesAnuales1977_2024.csv"}
for df_name,file_name in df_to_save.items():
    # Crea la ruta
    file_path = os.path.join(directory_path, file_name)
    # Crea el directorio si no existe
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    # Guarda el Dataframe en la ruta especificada
    globals()[df_name].to_csv(file_path, index=False)
    print(f"DataFrame guardado en: {file_path}")

Indicar la ruta de Google Drive para almacenar el archivo: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/scrapping_RankingCine200.csv
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/scrapping_FranquiciasReleases.csv
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/scrapping_CBM_RecaudacionDiaria.csv
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/scrapping_RecaudacionDiaria10.csv
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/scrapping_RecaudacionFeriados.csv
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/scrapping_CBM_Releases.csv
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/Data/scrapping_PrecioEntradas1977_2025.csv
DataFrame guardado en: /content/dri

In [ ]:
'''
# Solicita el directorio de Google Drive para guardar el archivo
directory_path = input("Indicar la ruta de Google Drive para almacenar el archivo: ")

# Crea la ruta
file_path = os.path.join(directory_path, 'ranking_cine.csv')

# Crea el directorio si no existe
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Guarda el Dataframe en la ruta especificada
dfranking_cine_depurado.to_csv(file_path, index=False)

print(f"DataFrame guardado en: {file_path}")
'''

Indicar la ruta de Google Drive para almacenar el archivo: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/dfranking_cine_depurado.csv


In [ ]:
'''
# Crea la ruta
file_path = os.path.join(directory_path, 'indices_anuales.csv')

# Crea el directorio si no existe
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Guarda el Dataframe en la ruta especificada
df_indices.to_csv(file_path, index=False)

print(f"DataFrame guardado en: {file_path}")
'''

Indicar la ruta de Google Drive para almacenar el archivo: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/df_indices.csv


In [ ]:
'''
# Crea la ruta
file_path = os.path.join(directory_path, 'franquicias_releases.csv')

# Crea el directorio si no existe
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Guarda el Dataframe en la ruta especificada
franquicias_releases.to_csv(file_path, index=False)

print(f"DataFrame guardado en: {file_path}")
'''

Indicar la ruta de Google Drive para almacenar el archivo: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo
DataFrame guardado en: /content/drive/MyDrive/Colab Notebooks/Datos/BoxOfficeMojo/franquicias_releases.csv


In [ ]:
'''
# Crea la ruta
file_path = os.path.join(directory_path, 'recaudacion_diaria.csv')

# Crea el directorio si no existe
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Guarda el Dataframe en la ruta especificada
franquicias_releases.to_csv(file_path, index=False)

print(f"DataFrame guardado en: {file_path}")
'''

In [ ]:
'''
# Crea la ruta
file_path = os.path.join(directory_path, 'recaudacion_feriados.csv')

# Crea el directorio si no existe
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Guarda el Dataframe en la ruta especificada
franquicias_releases.to_csv(file_path, index=False)

print(f"DataFrame guardado en: {file_path}")
'''